In [1]:
#creating State Summary Data

In [2]:
import pandas as pd
import numpy as np

In [3]:
#load and summarize the Wage and Cost Data
CostDF = pd.read_csv("CostDFwithPop.csv",index_col=0)
WageDF = pd.read_csv("WageDFwithPop.csv",index_col=0)

In [4]:
#WageDF.where(WageDF['State'] == 'West Virginia').dropna(subset='State')
WageDF['State'].replace('Virginia','East Virginia', inplace=True)
CostDF['State'].replace('Virginia','East Virginia', inplace=True)
#WageDF.where(WageDF['State'] == 'East Virginia').dropna(subset='State')

In [5]:
StateNames = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
              'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
              'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
              'Maine', 'Maryland', 'Massachusetts','Michigan', 'Minnesota',
              'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
              'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina',
              'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
              'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 
              'Utah', 'Vermont', 'East Virginia', 'Washington', 'West Virginia', 
              'Wisconsin', 'Wyoming']

In [6]:
NewCostDF = pd.DataFrame(columns = CostDF.columns)
NewCostDF.drop(['County','2022 Population Est'],axis=1,inplace=True)
for state in StateNames:
    Dfstate = CostDF.where(CostDF['State'] == state).dropna(subset='State')
    StatePopSum = Dfstate.copy().drop_duplicates('County')
    SPSum = np.sum(StatePopSum['2022 Population Est'])
    #if state == 'West Virginia':
    #    print(SPSum)
    CountyList = set(list(Dfstate['County']))
    NewStateDF = pd.DataFrame(columns = Dfstate.columns)
    NewStateDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
    for county in CountyList:
        countyweight = StatePopSum.where(StatePopSum['County'] == county).dropna(subset='County')['2022 Population Est']
        countyweight = float(countyweight/SPSum)
        #if state == 'West Virginia':
        #    print(countyweight)
        countyDF = Dfstate.where(Dfstate['County'] == county).dropna(subset='County')
        countyDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
        toconcat = pd.DataFrame(columns = countyDF.columns)
        countyDF.set_index('CostType',inplace=True)
        for column in countyDF.columns:
            countyDF[column] = countyDF[column].apply(lambda x: str(x[1:]))
            countyDF[column] = countyDF[column].str.replace(',', '')
            countyDF[column] = countyDF[column].apply(lambda x: int(x)*countyweight)
        countyDF.reset_index(inplace=True) #was automatically named CostType
        toconcat = pd.concat([toconcat,countyDF])
        #print(toconcat)
        NewStateDF = pd.concat([NewStateDF,toconcat])
        #break
    NewStateDF = NewStateDF.groupby('CostType').sum()
    NewStateDF.reset_index(inplace=True)
    #noticed that middle group of 4 was missing the data that was the same in the left and right groups of 4:
    NewStateDF.at[1,'2 ADULTS(1 WORKING) - 1'] = NewStateDF.at[1,'1 ADULT - 1']
    NewStateDF.at[1,'2 ADULTS(1 WORKING) - 2'] = NewStateDF.at[1,'1 ADULT - 2']
    NewStateDF.at[1,'2 ADULTS(1 WORKING) - 3'] = NewStateDF.at[1,'1 ADULT - 3']
    NewStateDF['State'] = state
    #if state == 'West Virginia':
    #    print(NewStateDF)
    NewCostDF = pd.concat([NewCostDF,NewStateDF])

In [7]:
NewCostDF.where(NewCostDF['State'] == 'West Virginia').dropna(subset='State')

,CostType,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State
0,Annual taxes,5938.910945,13225.287512,16919.978353,22381.144215,10231.812632,12358.458777,14846.77095,16075.326679,10231.812632,14737.947369,19328.057343,22797.144215,West Virginia
1,Child Care,0.0,8715.993188,17432.438781,26148.287734,0.0,8715.993188,17432.438781,26148.287734,0.0,8715.993188,17432.438781,26148.287734,West Virginia
2,Civic,2503.0,4019.0,4965.0,6483.0,4019.0,4965.0,6483.0,5153.0,4019.0,4965.0,6483.0,5153.0,West Virginia
3,Food,3351.0,4937.0,7417.0,9838.0,6143.0,7645.0,9856.0,11999.0,6143.0,7645.0,9856.0,11999.0,West Virginia
4,Housing,7657.144318,10027.68896,10027.68896,13206.30674,8180.221609,10027.68896,10027.68896,13206.30674,8180.221609,10027.68896,10027.68896,13206.30674,West Virginia
5,Medical,2599.0,7207.0,6994.0,7303.0,6092.0,6994.0,7303.0,7092.0,6092.0,6994.0,7303.0,7092.0,West Virginia
6,Other,4508.0,6723.0,6150.0,8948.0,6723.0,6150.0,8948.0,8655.0,6723.0,6150.0,8948.0,8655.0,West Virginia
7,Required annual income after taxes,26250.217264,51528.668342,65818.420221,87062.095076,41054.941013,48613.68896,57753.675154,62532.322914,41054.941013,57329.682148,75186.113934,88680.610648,West Virginia
8,Required annual income before taxes,32189.112957,64753.906503,82738.579294,109443.386267,51286.773737,60972.152582,72600.394405,78607.432072,51286.773737,72067.75284,94514.083349,111477.848061,West Virginia
9,Transportation,5509.0,9776.0,12709.0,15012.0,9776.0,12709.0,15012.0,16304.0,9776.0,12709.0,15012.0,16304.0,West Virginia


In [8]:
NewWageDF = pd.DataFrame(columns = WageDF.columns)
NewWageDF.drop(['County','2022 Population Est'],axis=1,inplace=True)
for state in StateNames:
    Dfstate = WageDF.where(WageDF['State'] == state).dropna(subset='State')
    StatePopSum = Dfstate.copy().drop_duplicates('County')
    SPSum = np.sum(StatePopSum['2022 Population Est'])
    CountyList = set(list(Dfstate['County']))
    NewStateDF = pd.DataFrame(columns = Dfstate.columns)
    NewStateDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
    for county in CountyList:
        countyweight = StatePopSum.where(StatePopSum['County'] == county).dropna(subset='County')['2022 Population Est']
        countyweight = float(countyweight/SPSum)
        countyDF = Dfstate.where(Dfstate['County'] == county).dropna(subset='County')
        countyDF.drop(['State','County','2022 Population Est'],axis=1,inplace=True)
        toconcat = pd.DataFrame(columns = countyDF.columns)
        countyDF.set_index('WageType',inplace=True)
        for column in countyDF.columns:
            countyDF[column] = countyDF[column].apply(lambda x: str(x[1:]))
            countyDF[column] = countyDF[column].str.replace(',', '')
            countyDF[column] = countyDF[column].apply(lambda x: float(x.strip())*countyweight)
        countyDF.reset_index(inplace=True)
        toconcat = pd.concat([toconcat,countyDF])
        NewStateDF = pd.concat([NewStateDF,toconcat])
        #break
    NewStateDF = NewStateDF.groupby('WageType').sum()
    NewStateDF.reset_index(inplace=True)
    NewStateDF['State'] = state
    NewWageDF = pd.concat([NewWageDF,NewStateDF])

In [9]:
NewWageDF.where(NewWageDF['State'] == 'West Virginia').dropna(subset='State')

,WageType,1 ADULT - 0,1 ADULT - 1,1 ADULT - 2,1 ADULT - 3,2 ADULTS(1 WORKING) - 0,2 ADULTS(1 WORKING) - 1,2 ADULTS(1 WORKING) - 2,2 ADULTS(1 WORKING) - 3,2 ADULTS(BOTH WORKING) - 0,2 ADULTS(BOTH WORKING) - 1,2 ADULTS(BOTH WORKING) - 2,2 ADULTS(BOTH WORKING) - 3,State
0,Living Wage,15.475221,31.131661,39.776676,52.617577,24.657034,29.316051,34.904471,37.791463,12.327845,17.323719,22.721106,26.797859,West Virginia
1,Minimum Wage,8.75,8.75,8.75,8.75,8.75,8.75,8.75,8.75,8.75,8.75,8.75,8.75,West Virginia
2,Poverty Wage,6.19,8.38,10.56,12.74,8.38,10.56,12.74,14.92,4.19,5.28,6.37,7.46,West Virginia


In [10]:
NewWageDF['State'].replace('East Virginia','Virginia', inplace=True)
NewCostDF['State'].replace('East Virginia','Virginia', inplace=True)

In [11]:
NewWageDF.to_csv('StateWageDF.csv')
NewCostDF.to_csv('StateCostDF.csv')